In [5]:
import gymnasium as gym
import numpy as npy
import random as rng

In [6]:
prueba = gym.make('Blackjack-v1', natural=False, sab=False, render_mode = "human")

#Métodos:
#reset(self) : Reinicia el estado del entorno, a su estado inicial, devolviendo una observación de dicho estado.
#step(self, action) : "Avanza" un timestep. Devuelve: observation, reward, done, info.
#render(self) : Muestra en pantalla el entorno.
#close(self) : Finaliza con la instancia del agente.
#seed(self) : Establece la semilla aleatoria del generador de números aleatorios del presente entorno.

#Atributos:
#action_space : El objeto de tipo Space correspondiente al espacio de acciones válidas.
#observation_space : El objeto de tipo Space correspondiente a todos los rangos posibles de observaciones.
#reward_range : Tupla que contiene los valores mínimo y máximo de recompensa posible.


In [ ]:
print("Tamaño de espacio de estados", prueba.observation_space)
print("Estado aleatorio", prueba.observation_space.sample())
size_estados = prueba.observation_space[0].n * prueba.observation_space[1].n * prueba.observation_space[2].n
print("Hay", size_estados, " estados posibles.")
print("Acciones posibles", prueba.action_space.n)
print("Acción aleatoria", prueba.action_space.sample())
size_acciones = prueba.action_space.n
print("Hay", size_acciones, "acciones posibles.")

Tamaño de espacio de estados Tuple(Discrete(32), Discrete(11), Discrete(2))
Estado aleatorio (np.int64(27), np.int64(8), np.int64(0))
Hay 704  estados posibles.
Acciones posibles 2
Acción aleatoria 1
Hay 2  acciones posibles.


In [8]:
# Reseteamos el entorno y obtenemos el estado inicial
#estado = prueba.reset()

# Imprime el estado inicial
# print("Estado inicial:", estado)

# # Simulamos una acción para ver cómo se actualiza el estado
# accion = 1  # Por ejemplo, podemos elegir 'hit'
# nuevo_estado, recompensa, hecho, info, a = prueba.step(accion)

# print("Nuevo estado:", nuevo_estado)
# print("Recompensa:", recompensa)
# print("Terminado:", hecho)
# print("Info:", info)

In [9]:
# for i_episode in range(20):
#     observation = prueba.reset()
#     for t in range(1000):
#         prueba.render()
#         #print(observation)
#         action = prueba.action_space.sample()
#         print(action)
#         observation, reward, done, info,_  = prueba.step(action)
        
#         if done:
#             print("Episode finished after {} timesteps".format(t+1))
#             break
# prueba.close()

In [10]:
env = gym.make('Blackjack-v1', natural=False, sab=False)
#env = gym.make('Blackjack-v1', natural=False, sab=False, render_mode = "human")

tamObservaciones = env.observation_space[0].n * env.observation_space[1].n * env.observation_space[2].n
tamAcciones = env.action_space.n

#Inicializo toda la Matriz Q-Learning a 0 (Dimensiones de 704x2)
matrizQ = npy.zeros([tamObservaciones, tamAcciones])


In [11]:
a = 0.5 #Valor de alpha (α) 
y = 0.8 #Valor de gamma (γ)
e = 0.8 #Valor de epsilon (ε)

episodios = 15 #Numero de simulaciones que se van a ejecutar
minSuma = 4 #Valor necesario para calcular las posiciones en la matriz
ourMod = 1 #Modificador auxiliar por si queremos aumentar o disminuir la recompensa predeterminada
castigos = 0 #Numero de penalizaciones que se han recibido

for i in range(episodios):

    print("Simulacion Nº" + str(i) + "\n")
    estado = env.reset()
    pos = (estado[0][1] - 1) + 20 * (estado[0][0] - minSuma) + 10 * estado[0][2]
    done = False
    print("Estado inicial: " + str(estado[0]))
    print("Posicion en la matriz: " + str(pos))
    print("··························")

    while not done:
        #env.render()
        if(rng.uniform(0, 1) > e): #Exploracion
            accion = env.action_space.sample()
        else: #Explotacion
            accion = npy.argmax(matrizQ[pos]) 

        estadoNext, recompensa, done, info, info2 = env.step(accion)

        recompensa *= ourMod
        posNext = (estadoNext[1] - 1) + 20 * (estadoNext[0] - minSuma) + 10 * estadoNext[2]

        valorAntiguo = matrizQ[pos, accion]
        siguienteMax = npy.max(matrizQ[posNext])

        valorNuevo = (1 - a) * valorAntiguo + a * (recompensa + y * siguienteMax)
        matrizQ[pos, accion] = valorNuevo
        if recompensa < 0: castigos += 1

        if accion == 0: print("Planta") 
        else: print("Pide")
        print("Estado siguiente: " + str(estadoNext))
        print("Posicion en la matriz: " + str(posNext))
        print("Recompensa: " + str(recompensa))
        print("Valor previo a la accion: " + str(valorAntiguo))
        print("Valor posterior a la accion: " + str(valorNuevo))

        estado = estadoNext
        pos = posNext
        if not done: print("··························")
            
    print("\n========================================\n")

#env.close()


Simulacion Nº0

Estado inicial: (13, 5, 0)
Posicion en la matriz: 184
··························
Planta
Estado siguiente: (13, 5, 0)
Posicion en la matriz: 184
Recompensa: -1.0
Valor previo a la accion: 0.0
Valor posterior a la accion: -0.5


Simulacion Nº1

Estado inicial: (12, 2, 0)
Posicion en la matriz: 161
··························
Planta
Estado siguiente: (12, 2, 0)
Posicion en la matriz: 161
Recompensa: 1.0
Valor previo a la accion: 0.0
Valor posterior a la accion: 0.5


Simulacion Nº2

Estado inicial: (20, 10, 0)
Posicion en la matriz: 329
··························
Planta
Estado siguiente: (20, 10, 0)
Posicion en la matriz: 329
Recompensa: 1.0
Valor previo a la accion: 0.0
Valor posterior a la accion: 0.5


Simulacion Nº3

Estado inicial: (7, 10, 0)
Posicion en la matriz: 69
··························
Planta
Estado siguiente: (7, 10, 0)
Posicion en la matriz: 69
Recompensa: -1.0
Valor previo a la accion: 0.0
Valor posterior a la accion: -0.5


Simulacion Nº4

Estado inicial: 

In [12]:
print("Numero de castigos finales: " + str(castigos))
for i in range (tamObservaciones):
    print("Fila Nº" + str(i) + " " + str(matrizQ[i]))

Numero de castigos finales: 8
Fila Nº0 [0. 0.]
Fila Nº1 [0. 0.]
Fila Nº2 [0. 0.]
Fila Nº3 [0. 0.]
Fila Nº4 [0. 0.]
Fila Nº5 [0. 0.]
Fila Nº6 [0. 0.]
Fila Nº7 [0. 0.]
Fila Nº8 [0. 0.]
Fila Nº9 [0. 0.]
Fila Nº10 [0. 0.]
Fila Nº11 [0. 0.]
Fila Nº12 [0. 0.]
Fila Nº13 [0. 0.]
Fila Nº14 [0. 0.]
Fila Nº15 [0. 0.]
Fila Nº16 [0. 0.]
Fila Nº17 [0. 0.]
Fila Nº18 [0. 0.]
Fila Nº19 [0. 0.]
Fila Nº20 [0. 0.]
Fila Nº21 [0. 0.]
Fila Nº22 [0. 0.]
Fila Nº23 [0. 0.]
Fila Nº24 [0. 0.]
Fila Nº25 [0. 0.]
Fila Nº26 [0. 0.]
Fila Nº27 [0. 0.]
Fila Nº28 [0. 0.]
Fila Nº29 [0. 0.]
Fila Nº30 [0. 0.]
Fila Nº31 [0. 0.]
Fila Nº32 [0. 0.]
Fila Nº33 [0. 0.]
Fila Nº34 [0. 0.]
Fila Nº35 [0. 0.]
Fila Nº36 [0. 0.]
Fila Nº37 [0. 0.]
Fila Nº38 [0. 0.]
Fila Nº39 [0. 0.]
Fila Nº40 [0. 0.]
Fila Nº41 [0. 0.]
Fila Nº42 [0. 0.]
Fila Nº43 [0. 0.]
Fila Nº44 [0. 0.]
Fila Nº45 [0. 0.]
Fila Nº46 [0. 0.]
Fila Nº47 [0. 0.]
Fila Nº48 [0. 0.]
Fila Nº49 [0. 0.]
Fila Nº50 [0. 0.]
Fila Nº51 [0. 0.]
Fila Nº52 [0. 0.]
Fila Nº53 [0. 0.]
Fila Nº5